<a href="https://colab.research.google.com/github/YuriSerhiienko/data_science_hw10/blob/main/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [8]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Нормалізація зображень
train_images = train_images / 255.0
test_images = test_images / 255.0

# Зміна розмірів зображень для CNN
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nТочність на тестових даних:', test_acc)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)

print("\nМатриця помилок:")
print(confusion_matrix(test_labels, y_pred))

print("\nКласифікаційний звіт:")
print(classification_report(test_labels, y_pred))


Epoch 1/10
1500/1500 [==============================] - 9s 5ms/step - loss: 0.4733 - accuracy: 0.8285 - val_loss: 0.3581 - val_accuracy: 0.8702
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3163 - accuracy: 0.8858 - val_loss: 0.3229 - val_accuracy: 0.8827
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2699 - accuracy: 0.9004 - val_loss: 0.2833 - val_accuracy: 0.8954
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2348 - accuracy: 0.9135 - val_loss: 0.2550 - val_accuracy: 0.9048
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2121 - accuracy: 0.9210 - val_loss: 0.2574 - val_accuracy: 0.9075
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1893 - accuracy: 0.9301 - val_loss: 0.2638 - val_accuracy: 0.9098
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.1671 - accuracy: 0.9383 - val_loss: 0.2655 - val_accuracy:

In [9]:
# Завантаження та підготовка датасету
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# Зміна розмірів зображень для VGG16
def resize_images(images):
    images_resized = np.array([tf.image.resize(img, (32, 32)) for img in images])
    return images_resized

train_images_resized = resize_images(train_images[..., np.newaxis])
test_images_resized = resize_images(test_images[..., np.newaxis])

# Конвертація зображень у формат RGB
train_images_rgb = np.repeat(train_images_resized, 3, axis=-1)
test_images_rgb = np.repeat(test_images_resized, 3, axis=-1)

# Завантаження VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg_base.trainable = False  # Заморожування ваг

# Створення моделі
model = tf.keras.Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(train_images_rgb, train_labels, epochs=10, validation_split=0.2)

# Оцінка моделі
test_loss, test_acc = model.evaluate(test_images_rgb, test_labels)
print('\nТочність на тестових даних:', test_acc)

# Передбачення та метрики якості
predictions = model.predict(test_images_rgb)
y_pred = np.argmax(predictions, axis=1)

print("\nМатриця помилок:")
print(confusion_matrix(test_labels, y_pred))

print("\nКласифікаційний звіт:")
print(classification_report(test_labels, y_pred))


Epoch 1/10
1500/1500 [==============================] - 16s 10ms/step - loss: 0.6873 - accuracy: 0.7614 - val_loss: 0.4779 - val_accuracy: 0.8272
Epoch 2/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.5048 - accuracy: 0.8208 - val_loss: 0.4380 - val_accuracy: 0.8355
Epoch 3/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.4679 - accuracy: 0.8307 - val_loss: 0.4253 - val_accuracy: 0.8438
Epoch 4/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.4441 - accuracy: 0.8383 - val_loss: 0.4018 - val_accuracy: 0.8513
Epoch 5/10
1500/1500 [==============================] - 14s 10ms/step - loss: 0.4279 - accuracy: 0.8443 - val_loss: 0.4049 - val_accuracy: 0.8505
Epoch 6/10
1500/1500 [==============================] - 14s 10ms/step - loss: 0.4169 - accuracy: 0.8453 - val_loss: 0.3918 - val_accuracy: 0.8539
Epoch 7/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.4092 - accuracy: 0.8495 - val_loss: 0.3932 -

Попередня робота (Багатошаровий перцептрон)

Архітектура: Базувалася на повнозв'язних шарах (Dense layers).

Точність на тестових даних: 88.08% у базовій моделі, 86.93% після оновлення архітектури.

Інші метрики: Висока точність (precision) та повнота (recall), особливо після оновлення архітектури.

Нинішня робота (Згорткова нейронна мережа, CNN)

Архітектура: Базувалася на згорткових шарах (Conv2D), що є типовим для задач обробки зображень.

Точність на тестових даних: 90.88%.

Інші метрики: Деталізовані метрики (матриця помилок, точність, повнота, F1-оцінка) показали високу ефективність моделі у класифікації зображень.

Висновки

Ефективність архітектури: Згорткова нейронна мережа (CNN) показала кращі результати для задачі класифікації зображень у порівнянні з багатошаровим перцептроном. Це очікувано, оскільки CNN краще адаптовані для виявлення шаблонів у зображеннях.

Підхід до обробки даних: CNN використовує просторові відносини між пікселями, що дозволяє ефективно виділяти особливості зображень, в той час як багатошаровий перцептрон обробляє дані як одновимірний масив.



In [10]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Нормалізація зображень
train_images = train_images / 255.0
test_images = test_images / 255.0

# Перетворення міток у one-hot вектори
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_images = np.array([np.resize(img, (32, 32, 3)) for img in train_images])
test_images = np.array([np.resize(img, (32, 32, 3)) for img in test_images])

vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg_base.trainable = False  # Заморожування ваг VGG16

model = tf.keras.Sequential([
    vgg_base,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nТочність на тестових даних:', test_acc)

predictions = model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)
test_labels_argmax = np.argmax(test_labels, axis=1)

print("\nМатриця помилок:")
print(confusion_matrix(test_labels_argmax, y_pred))

print("\nКласифікаційний звіт:")
print(classification_report(test_labels_argmax, y_pred))


Epoch 1/10
1500/1500 [==============================] - 16s 10ms/step - loss: 1.1173 - accuracy: 0.6127 - val_loss: 0.8467 - val_accuracy: 0.7089
Epoch 2/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.8862 - accuracy: 0.6960 - val_loss: 0.7723 - val_accuracy: 0.7339
Epoch 3/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.8249 - accuracy: 0.7168 - val_loss: 0.7343 - val_accuracy: 0.7442
Epoch 4/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.7943 - accuracy: 0.7263 - val_loss: 0.7319 - val_accuracy: 0.7462
Epoch 5/10
1500/1500 [==============================] - 17s 11ms/step - loss: 0.7696 - accuracy: 0.7346 - val_loss: 0.6986 - val_accuracy: 0.7552
Epoch 6/10
1500/1500 [==============================] - 17s 12ms/step - loss: 0.7530 - accuracy: 0.7380 - val_loss: 0.6845 - val_accuracy: 0.7622
Epoch 7/10
1500/1500 [==============================] - 15s 10ms/step - loss: 0.7420 - accuracy: 0.7438 - val_loss: 0.6729 -

In [11]:
# Завантаження та підготовка датасету
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Зміна розміру зображень до 32x32 та конвертація у формат RGB
train_images_rgb = np.array([cv2.cvtColor(cv2.resize(img, (32, 32)), cv2.COLOR_GRAY2RGB) for img in train_images]) / 255.0
test_images_rgb = np.array([cv2.cvtColor(cv2.resize(img, (32, 32)), cv2.COLOR_GRAY2RGB) for img in test_images]) / 255.0

# Перетворення міток у one-hot вектори
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Завантаження VGG16
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Розморожування останніх шарів VGG16
for layer in vgg_base.layers[:-4]:
    layer.trainable = False

# Створення кастомізованої моделі
model = Sequential([
    vgg_base,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Навчання моделі
history = model.fit(train_images_rgb, train_labels, epochs=20, batch_size=32, validation_split=0.2)

# Оцінка моделі
test_loss, test_acc = model.evaluate(test_images_rgb, test_labels)
print('\nТочність на тестових даних:', test_acc)

# Передбачення та отримання міток класів
predictions = model.predict(test_images_rgb)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(test_labels, axis=1)

# Матриця помилок
from sklearn.metrics import confusion_matrix, classification_report
print("\nМатриця помилок:")
print(confusion_matrix(true_classes, predicted_classes))

# Класифікаційний звіт
print("\nКласифікаційний звіт:")
print(classification_report(true_classes, predicted_classes))


Epoch 1/20
1500/1500 [==============================] - 28s 17ms/step - loss: 0.4968 - accuracy: 0.8253 - val_loss: 0.3554 - val_accuracy: 0.8725
Epoch 2/20
1500/1500 [==============================] - 26s 17ms/step - loss: 0.3229 - accuracy: 0.8868 - val_loss: 0.3089 - val_accuracy: 0.8898
Epoch 3/20
1500/1500 [==============================] - 26s 17ms/step - loss: 0.2801 - accuracy: 0.9003 - val_loss: 0.2894 - val_accuracy: 0.8962
Epoch 4/20
1500/1500 [==============================] - 26s 17ms/step - loss: 0.2540 - accuracy: 0.9077 - val_loss: 0.2917 - val_accuracy: 0.8920
Epoch 5/20
1500/1500 [==============================] - 25s 17ms/step - loss: 0.2336 - accuracy: 0.9151 - val_loss: 0.2654 - val_accuracy: 0.9082
Epoch 6/20
1500/1500 [==============================] - 28s 19ms/step - loss: 0.2107 - accuracy: 0.9246 - val_loss: 0.2746 - val_accuracy: 0.9050
Epoch 7/20
1500/1500 [==============================] - 28s 19ms/step - loss: 0.1986 - accuracy: 0.9293 - val_loss: 0.2592 -

CNN показала значно кращі результати в порівнянні з багатошаровою мережею. Точність CNN була вищою і становила близько 91%, в той час як точність багатошарової мережі була нижчою і становила близько 87%. Це означає, що згорткова мережа ефективніше впоралася з класифікацією зображень Fashion MNIST.

Висновок полягає в тому, що для обробки зображень, особливо для завдань класифікації зображень, CNN може бути кращим вибором порівняно з багатошаровими мережами. Використання згорткових шарів дозволяє моделі автоматично виявляти важливі ознаки на зображеннях і покращує її результати.